In [1]:
# first download the tinydiarize model (should be the same as in the tinydiarize example)
! /home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/download-ggml-model.sh medium.en-tdrz

# model is saved at /home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/ggml-small.en-tdrz.bin
# dont forget to `make` before trying to run inference

Invalid model: medium.en-tdrz

Available models:
  tiny tiny.en tiny-q5_1 tiny.en-q5_1
  base base.en base-q5_1 base.en-q5_1
  small small.en small.en-tdrz small-q5_1 small.en-q5_1
  medium medium.en medium-q5_0 medium.en-q5_0
  large-v1 large-v2 large-v2-q5_0 large-v3 large-v3-q5_0



In [5]:
# run diarization via tinydiarize but from whisper.cpp
# -m as model and -f as file and -tdrz to enable diarization -sow
# vram usage is around 2.8 gigs (probably only half of it is from whisper.cpp)
! /home/kozi/Documents/_onlab_git/ami/whisper.cpp//main -m '/home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/ggml-small.en-tdrz.bin' -f '/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/audio/Array1-01/EN2002a.Array1-01.wav' -tdrz

# view the results in ami_nemo_benchmark.ipynb

whisper_init_from_file_with_params_no_state: loading model from '/home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/ggml-small.en-tdrz.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 0
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 3 (small)
whisper_model_load: adding 1608 extra tokens
whisper_model_load: n_langs       = 99
whisper_model_load

In [ ]:
!python3 /home/kozi/Documents/_onlab_git/dscore/score.py -s '/home/kozi/Documents/_onlab_git/ami/audio-test-pyannote.rttm' -r "/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/rttm/EN2002a.rttm" # system outputs and reference

In [6]:
# try out -sow alias --split-on-words
! /home/kozi/Documents/_onlab_git/ami/whisper.cpp//main -m '/home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/ggml-small.en-tdrz.bin' -f '/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/audio/Array1-01/EN2002a.Array1-01.wav' -tdrz -sow
# does not really seem to do anything it should

whisper_init_from_file_with_params_no_state: loading model from '/home/kozi/Documents/_onlab_git/ami/whisper.cpp/models/ggml-small.en-tdrz.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 0
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 3 (small)
whisper_model_load: adding 1608 extra tokens
whisper_model_load: n_langs       = 99
whisper_model_load

In [ ]:
# stolen from tinydiarize_test.ipynb; this is not used here                                         
def determine_speaker(idx):                                                    
    return idx % 2  # Assuming two speakers indexed as 0 and 1                 
                                                                            
# Create RTTM lines                                                            
rttm_lines = []                                                                
txt_lines = []                                                                 
for idx, segment in enumerate(result['segments']):                               
    speaker = determine_speaker(idx)                                           
    start = segment['start']                                                   
    end = segment['end']                                                       
    duration = end - start                                                     
    text = segment['text'].replace(' [SPEAKER TURN]', '').strip()              
                                                                            
    # RTTM line format                                                         
    rttm_line = f"SPEAKER EN2002a 1 {start:.3f} {duration:.3f} <NA> <NA> speaker{speaker} <NA> <NA>"
    rttm_lines.append(rttm_line)                                               
                                                                            
    # TXT line format                                                          
    txt_line = f"Speaker {speaker}: {text}"                                    
    txt_lines.append(txt_line)                                                 
                                                                            
# Write RTTM file                                                              
with open('/home/kozi/Documents/_onlab_git/ami/tinydiarize_output.rttm', 'w') as rttm_file:
    for line in rttm_lines:                                                    
        rttm_file.write(line + '\n')

In [8]:
# convert the output text to an rttm file
import re                                                                                                       
from datetime import datetime, timedelta                                                                        
                                                                                                                
                                                                                                                
def parse_timestamp(timestamp):                                                                                 
    """Convert timestamp in format [hh:mm:ss.sss] to seconds."""                                                
    time_part = timestamp.strip("[]")                                                                           
    time_obj = datetime.strptime(time_part, '%H:%M:%S.%f')                                                      
    seconds = timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second).total_seconds()  
    time_obj.microsecond / 1e6
    return seconds                                                                                              
                                                                                                                
                                                                                                                
def parse_input_text(input_text):                                                                               
    """Parse the input text and extract intervals and text segments."""                                         
    segments = input_text.split("\n")                                                                           
    parsed_segments = []                                                                                        
                                                                                                                
    for segment in segments:                                                                                    
        if segment.strip():                                                                                     
            match = re.match(r'\[(.*?)\s*-->\s*(.*?)\]\s*(.*)\[SPEAKER_TURN\]', segment.strip())                
            if match:                                                                                           
                start_time, end_time, text = match.groups()                                                     
                start_time = parse_timestamp(start_time)                                                        
                end_time = parse_timestamp(end_time)                                                            
                duration = end_time - start_time                                                                
                parsed_segments.append((start_time, duration, text.strip()))                                    
                                                                                                                
    return parsed_segments                                                                                      
                                                                                                                
                                                                                                                
def generate_rttm(parsed_segments):                                                                             
    """Generate RTTM formatted output from parsed segments."""                                                  
    rttm_lines = []                                                                                             
    speaker_id = 0                                                                                              
                                                                                                                
    for start_time, duration, text in parsed_segments:                                                          
        rttm_line = f"""SPEAKER EN2002a 1 {start_time:.3f} {duration:.3f} <NA> <NA> speaker{speaker_id} <NA> <NA>"""
        rttm_lines.append(rttm_line)                                                                            
        speaker_id = (speaker_id + 1) % 2  # Alternate between two speakers (speaker0, speaker1)
    return "\n".join(rttm_lines)
                                                                                                                
                                                                                                                
# The provided text that should be transformed                                                                  
input_text = '''
[00:00:00.140 --> 00:00:03.660]   I wonder how much of the meetings is talking about the stuff at the meetings? [SPEAKER_TURN]
[00:00:03.660 --> 00:00:27.120]   Yeah exactly, yeah yeah, I know. Welcome start. Okay. So shall we should we just go around and everyone says what they what they've been doing, what what what I'm saying. Yeah, okay. [SPEAKER_TURN]
[00:00:27.120 --> 00:00:29.460]   Sorry.
[00:00:29.820 --> 00:00:34.680]   Okay. Well I've got a browser now which, whoops, I already know. [SPEAKER_TURN]
[00:00:34.680 --> 00:00:36.160]   What was that? [SPEAKER_TURN]
[00:00:36.160 --> 00:00:36.720]   Mm-hmm. [SPEAKER_TURN]
[00:00:36.720 --> 00:00:39.200]   Oh I dropped mine as well. [SPEAKER_TURN]
[00:00:39.200 --> 00:00:43.440]   Give it a s [SPEAKER_TURN]
[00:00:43.440 --> 00:00:45.060]   It's definitely a good start. [SPEAKER_TURN]
[00:00:45.060 --> 00:00:49.060]   Okay. [SPEAKER_TURN]
[00:00:49.060 --> 00:00:50.100]   So yeah. [SPEAKER_TURN]
[00:00:50.420 --> 00:01:06.780]   So yeah, we've got a browser we've um which comes up automatically with the transcription box and the topics. And then when you go on the menu you can select a summarisation box which pops up and an audio player. [SPEAKER_TURN]
[00:01:06.780 --> 00:01:08.600]   Right. [SPEAKER_TURN]
[00:01:08.780 --> 00:01:24.800]   Um and that's I think the search works as well, so you can pop up a search. And it loads up um just the background window, so empty and um so when you start you have to either open open a particular observation or do a search and open it through that.
[00:01:24.800 --> 00:01:27.140]   That makes sense?
[00:01:28.600 --> 00:01:43.960]   Um the transcription box has got a summarise bo button which doesn't do anything yet. Um now I wonder how we wanna do you know we wanna pop up window with a speaker characterisation. [SPEAKER_TURN]
[00:01:43.960 --> 00:01:46.800]   We can do it. [SPEAKER_TURN]
[00:01:46.960 --> 00:02:07.320]   Either we could do that, you know when does it pop up come, either we can when you click on the the I_D_ any sort of I_D_ in the transcription box or we can put an extra button extra few buttons and extra summarise button so that you actually in the transcription box but so that you click on a button and then it opens whatever that speaker characterisation is.
[00:02:07.320 --> 00:02:14.860]   The problem is the the left click is already used because it highlights that um
[00:02:16.620 --> 00:02:23.680]   that part speech or whatever that what he's been saying, yeah. The left click highlights it. So that you can [SPEAKER_TURN]
[00:02:23.680 --> 00:02:31.760]   I don't know what happened when you double click actually, will that be a bit annoying if you have left click for one thing and double click for something else? [SPEAKER_TURN]
[00:02:31.760 --> 00:02:32.880]   Mm-hmm. [SPEAKER_TURN]
[00:02:32.880 --> 00:02:40.700]   Um I think so yeah when you left click it you can for example set set the marker there so that the the audio goes from there I think.
[00:02:42.680 --> 00:02:56.760]   Um so we can't put it on left click. We could put it on a right click. We could for example have a little menu that pops up. So right click on it menu, you know and, you can click on speaker characterisation on it pops up in there. [SPEAKER_TURN]
[00:02:56.760 --> 00:02:59.880]   Menu. [SPEAKER_TURN]
[00:02:59.880 --> 00:03:03.200]   That's a good point. I don't know. [SPEAKER_TURN]
[00:03:03.200 --> 00:03:07.040]   Otherwise you just have a right click on it you, know. [SPEAKER_TURN]
[00:03:07.040 --> 00:03:12.440]   I don't know about you, but i usually in Windows you right click, doesn't do anything, doesn't it opens a menu. [SPEAKER_TURN]
[00:03:12.560 --> 00:03:18.260]   Yeah, so. Yeah. So but that'd be yeah, I think it'd just yeah. Yeah. [SPEAKER_TURN]
[00:03:18.260 --> 00:03:24.320]   Same sort of idiom throughout kind of thing. So yeah, mm-hmm. [SPEAKER_TURN]
[00:03:24.320 --> 00:03:26.680]   Um or just you know or just a button. [SPEAKER_TURN]
[00:03:26.680 --> 00:03:40.120]   I guess a but um button makes a bit more sense 'cause otherwise you don't really know that oh, what if I right click that, what happens then? It's like yeah, it's for sure. [SPEAKER_TURN]
[00:03:40.120 --> 00:03:52.600]   Yeah, the menu pop up that. Well oh yeah, it's more obvious isn't, it? Maybe it could do. Yeah. Yeah, that's true yeah, it's more and true. But well, yes. [SPEAKER_TURN]
[00:03:52.600 --> 00:04:10.100]   Yeah. [SPEAKER_TURN]
[00:04:10.160 --> 00:04:26.800]   Um but actually what we could do is on in that menu as well we could have an option that pops up a window with all the all the meetings that that user has been in. Like a search for that user. Is that gonna be useful or too much or? [SPEAKER_TURN]
[00:04:26.800 --> 00:04:30.800]   Mm-hmm. [SPEAKER_TURN]
[00:04:30.800 --> 00:04:39.200]   Yeah. So when you right click on it one one option will be give me all your meetings characterise the speaker.
[00:04:40.100 --> 00:04:40.500]   Yep. [SPEAKER_TURN]
[00:04:40.500 --> 00:05:03.620]   Or is that too much? I mean I don't know. I mean it's more like part of the browsing sort of thing more than the speaker characterisation I, think Steve mentioned. It's true, you know how it it it involved the speaker characters. It's true to be nice to have it in connect into connection. [SPEAKER_TURN]
[00:05:05.700 --> 00:05:18.180]   I think as well I mean speaker characterisation to connect into I don't know to just some way to sort of leverage that information. Like we have it, we're doing it, but what how does it help the browsing? Or what can it give us in the browsing? Yeah. [SPEAKER_TURN]
[00:05:18.180 --> 00:05:35.520]   It's like a nice thing. We don't we're not actually sure what the user wants for it, but it's interesting so, why not give it to him. I I think we should do it, I mean [SPEAKER_TURN]
[00:05:35.520 --> 00:06:04.280]   Yeah. I mean I guess maybe we could have that as another thing of like well if you get the speaker characterisation that someone's talks a lot in a certain topic and then if you I don't know click on that topic you'll get I think that's in that topic. And you have to have it there. Yeah, I mean that's that's still another level further. I mean we're not even there yet you, know. So you pop up the speaker characterisation yet. You you might have to first define what what happens when you click on a user. [SPEAKER_TURN]
[00:06:04.440 --> 00:06:27.520]   We have the speaker characterisation first but, we're crossing the other side. I don't know if we can do it yet. Yeah. Yeah. Alright. Well is that just from the main thing? Is that from any particular window? Like if we wanna sort of get like if we're on the users versus like a topic or something have, a like a platform for these various things to have. [SPEAKER_TURN]
[00:06:27.520 --> 00:06:32.440]   Mm-hmm.
[00:06:32.440 --> 00:06:46.540]   Mm-hmm. Yeah. So we could do that in a similar way. Do it right click as well. And have different names. Yeah. [SPEAKER_TURN]
[00:06:46.540 --> 00:06:48.120]   Okay. Only the speaker. [SPEAKER_TURN]
[00:06:48.120 --> 00:07:04.160]   So yeah. So we have basically two options of of browsing the meetings is by either um searching and opening individual observations and when then we have the interlinking by right click basically. [SPEAKER_TURN]
[00:07:04.480 --> 00:07:34.460]   Right. Actually yeah, like the another common thing, like if we have it so it's set up that um if you double click and it goes to a default thing on the menu, I mean that's kind of a typical kind of action that you have. If you double click on a particular topic or something, then it goes to sort of a like a default view, so it'll be like all the um cases of that topic, either whatever we decide the default to be, whether it's in all meetings or just in one meeting and just show up the in particular instances of that topic in the meeting. Like I just think that to have the default there [SPEAKER_TURN]
[00:07:34.460 --> 00:07:43.200]   So if somebody knows exactly what they wanna do and we know that that's the most common thing that's gonna happen, so the double click will not do that and just save them the trouble of right clicking and choosing the item on the menu. Yeah. [SPEAKER_TURN]
[00:07:56.960 --> 00:08:04.020]   have a default double click, so there's there's there'll be a choice for the menu as well. Yeah exactly, used double click. [SPEAKER_TURN]
[00:08:04.020 --> 00:08:13.880]   I'm not sure there's bigger characterization for instance for instance, but then then there's a problem with a lot of windows popping up again and they pop up without you even being quite sure what it is, you know. [SPEAKER_TURN]
[00:08:13.880 --> 00:08:15.660]   Yeah. [SPEAKER_TURN]
[00:08:15.660 --> 00:08:22.060]   But if you do double click on something, don't you expect something to happen then, [SPEAKER_TURN]
[00:08:22.060 --> 00:08:26.540]   Yeah maybe, you should have well [SPEAKER_TURN]
[00:08:26.900 --> 00:08:31.860]   Well I'll I'll then they close it. Yeah it's, not it's not going to load it. [SPEAKER_TURN]
[00:08:31.860 --> 00:08:53.860]   Especially if you have a touch pad. And then when she goes oh. Well I'll I'll check what it does on on the double click at the moment. 'Cause I think it might do something, but it's good stuff. But I don't I don't I really don't see anything obvious that that's, I mean if we can think of something like [SPEAKER_TURN]
[00:08:53.860 --> 00:08:56.520]   Yeah. [SPEAKER_TURN]
[00:08:57.120 --> 00:08:59.300]   Oh well. Sorry.
[00:08:59.300 --> 00:09:16.760]   I guess I can put some these. Yeah, what you're clicking on I, hope so. Um yeah another, thing single, audio files, do we wanna use them or not? [SPEAKER_TURN]
[00:09:16.760 --> 00:09:21.620]   We don't know yet do, we uh do we? Do you mean single channel? [SPEAKER_TURN]
[00:09:21.620 --> 00:09:25.280]   Yeah single, chan then single channel files, do we want them? I mean [SPEAKER_TURN]
[00:09:25.780 --> 00:09:31.200]   Probably not, but we might need them for disfluencies. If we can go to an audio file. But [SPEAKER_TURN]
[00:09:31.200 --> 00:09:49.640]   Somebody's talking over somebody else or something though.
[00:09:51.320 --> 00:10:03.360]   We'll just send them out for now and put it in by mail or something like that. Yeah Yeah. 'Cause otherwise it will they it will increase the amount. Don't you think we're gonna probably not going to the full print one, I think. [SPEAKER_TURN]
[00:10:03.360 --> 00:10:16.120]   Yeah w what do people think in general about the Windows clutter in there you've mentioned? Do you think you think it's gonna [SPEAKER_TURN]
[00:10:16.120 --> 00:10:20.880]   I think we probably need that though, to have the f the full flexibility we need to have that though. That's not what I mean. [SPEAKER_TURN]
[00:10:20.940 --> 00:10:26.360]   very much I w I wanna have everything customisable. Um [SPEAKER_TURN]
[00:10:26.360 --> 00:10:35.520]   Well the thing is typically though, is someone gonna have five Windows open at the same time? Probably not. Probably just two or two or three of whatever whatever, they're looking at like dialogue acts or whatever. [SPEAKER_TURN]
[00:10:49.800 --> 00:10:56.240]   I mean, you probably I don't know if you wanna have like the the full transcription and the summary at the same time.
[00:10:56.240 --> 00:11:03.440]   So maybe you can just like choose the s same window for transcription and summary. [SPEAKER_TURN]
[00:11:03.440 --> 00:11:06.760]   It's like have a tab there or something like that, you can just have a tab. [SPEAKER_TURN]
[00:11:06.960 --> 00:11:23.320]   Yeah tab, I think it's true. Oh that's right, I'm not sure. Yeah I, don't know if you'd say it's good. I think you're right. I'm sure yeah, I'm sure you could do anything. Yeah. [SPEAKER_TURN]
[00:11:23.320 --> 00:11:30.560]   I don't know if it's a bit of a cripple job as we're knowing talk at the A_ isn't sometimes it's a bit of a cripple. [SPEAKER_TURN]
[00:11:30.560 --> 00:11:32.840]   Mm. I can have a look. [SPEAKER_TURN]
[00:11:32.840 --> 00:11:34.460]   Yeah. [SPEAKER_TURN]
[00:11:34.840 --> 00:11:38.880]   Yeah, or just I don't know. But yeah that, might be an idea. [SPEAKER_TURN]
[00:11:38.880 --> 00:12:04.280]   Yeah.
[00:12:04.280 --> 00:12:05.140]   Press on which one? [SPEAKER_TURN]
[00:12:05.140 --> 00:12:07.360]   Or check where it's even happened to a whole bunch of people. [SPEAKER_TURN]
[00:12:07.360 --> 00:12:18.240]   Yeah, that'll reduce the number of um windows, wouldn't it, which is probably good. 'Cause otherwise Photoshop has a lot of windows. If you ever use Photoshop, it's got loads and you're always good for ever going, I'll get out of the way. There's always stuff in the ways, isn't there. [SPEAKER_TURN]
[00:12:18.240 --> 00:12:28.740]   Um well I don't mean people get used to it. I mean when you use uh Windows or Linux or whatever, you have a lot of windows open. You can manage them. [SPEAKER_TURN]
[00:12:28.740 --> 00:12:29.680]   Yeah. [SPEAKER_TURN]
[00:12:29.680 --> 00:12:32.060]   True. [SPEAKER_TURN]
[00:12:32.060 --> 00:12:34.240]   Yeah. Unless can you [SPEAKER_TURN]
[00:12:34.300 --> 00:12:42.160]   I don't suppose you can minimise them and stuff like that. Can can can you do that now? Okay. Well that's right, yeah. Yeah, of course you can actually, yes. [SPEAKER_TURN]
[00:12:42.160 --> 00:12:56.980]   Yeah. Like a lot of these can be like once we get the disfluencies out then we can you know transcript, with all the disfluency transcript with the disflu dysfluency. How about these you know how, all these sort of using the same window and so we can double-fire. And probably mutually um [SPEAKER_TURN]
[00:12:56.980 --> 00:13:02.780]   Yeah, that that was a plan for the disfluency anyway. So [SPEAKER_TURN]
[00:13:02.780 --> 00:13:03.840]   Yeah.
[00:13:04.020 --> 00:13:11.540]   Yeah, do you wanna do that with summaries then? Put the summaries in in in the same way now Mutually. exclusive things. [SPEAKER_TURN]
[00:13:11.540 --> 00:13:26.780]   Well, could I dunno, with transcripts it sounds reasonable to have like transcripts like maybe three alternative then, like full meeting, meeting without disfluencies and summaries. Yeah? Do you wanna do that? Yeah, yeah? [SPEAKER_TURN]
[00:13:26.780 --> 00:13:29.700]   Yeah, that kind of makes sense. Yeah. Yeah, so it's good.
[00:13:30.340 --> 00:13:31.680]   Mm-hmm.
[00:13:31.680 --> 00:13:56.560]   And finally the prototype piece of paper about What. kind of prototype could we produce? 'Cause I'm I'm just, you know, I go into the lab and I say, what am I gonna change today, you know? And it kind of just it just develops. I'm not aiming for anything. Do we wanna answer something? [SPEAKER_TURN]
[00:13:57.560 --> 00:13:58.880]   Um. [SPEAKER_TURN]
[00:13:58.880 --> 00:14:25.900]   I'm I'm pretty much getting there anyway, to be honest. So it's not gonna be a lot more uh I guess it is. I mean just to try and I mean for us to to be able to do something that we can include with your thing. Some some reason Yeah. Yeah. Mm-hmm. Yeah. Yeah, just so see the integration works. So we can do I think I'm trying to [SPEAKER_TURN]
[00:14:26.340 --> 00:14:30.960]   Um. So I guess that's what you're gonna take as well. [SPEAKER_TURN]
[00:14:30.960 --> 00:14:36.180]   I think that's kind of what he meant as well, isn't it? Like seeing it seeing it sort of start to do stuff, rather than just being [SPEAKER_TURN]
[00:14:36.180 --> 00:14:51.180]   I mean that was probably our intention, but we m might not have written it down. So I don't know. I don't know. [SPEAKER_TURN]
[00:14:51.240 --> 00:15:20.200]   Actually, is the code accessible, like the the gooey stuff that you've done there? Like can we just like copy like the latest stuff out of your directory or something like that? Like if we're gonna just try and like take a look or something. Try and just like and stuff. If you can if you can make it well readable, then anybody can see it. Yeah. Yeah. Yeah. Sure. I don't know if it's readable at the moment, but you can show up. You can cover it up. It's uh yeah. No, they're not well readable, they're not done they're, so they're running [SPEAKER_TURN]
[00:15:20.200 --> 00:15:27.080]   fill, they're running the fill. Then only you can read them. Like you can see them, but you can't read them. Okay, alright. [SPEAKER_TURN]
[00:15:27.080 --> 00:15:28.320]   Mm-hmm, yeah. [SPEAKER_TURN]
[00:15:28.320 --> 00:15:32.460]   Um so the directory is my number. [SPEAKER_TURN]
[00:15:32.540 --> 00:16:01.820]   Or you could just email to us. Okay, yeah, yeah, yeah. Right. Okay. Yeah, your number. Okay. Okay. Okay. Okay. And then it's and then um there's a folder um N_L_S_S_D_, and it's called the M_ browser. [SPEAKER_TURN]
[00:16:02.380 --> 00:16:16.020]   The M_ browser. Or it is. Um hope you hope you never have to look in my directory. It's just pure chaos. No, I'm I'm extremely old-fashioned. I've got balls, I think I. love balls. [SPEAKER_TURN]
[00:16:16.020 --> 00:16:22.020]   Mm-hmm. [SPEAKER_TURN]
[00:16:22.020 --> 00:16:29.420]   Yep, so that's me done. Someone else wanna talk about this stuff?
[00:16:29.420 --> 00:16:59.280]   Time to jump a once. I can tell you what I th I'm an S_P_T_. Oh um, I don't know. I basically I th started like browsing a bit just like trying to see if there's anything out there to use to sort of um just th what were you doing again. Yeah, so yeah hopefully, I was starting to play with this stuff. Yeah, yeah, let's get this stuff. Okay. [SPEAKER_TURN]
[00:16:59.360 --> 00:17:04.520]   I found a tutorial that uh inverted the search to indexing, but I haven't found yet. [SPEAKER_TURN]
[00:17:04.520 --> 00:17:10.840]   So you could probably just use that code. Like that was behind the question Chas number one. You know, the we're using code and stuff which is like okay. [SPEAKER_TURN]
[00:17:10.920 --> 00:17:21.640]   And that's pretty much fair, is it? Yeah well, I don't know if it's usable. I c I haven't looked at it. But but I found some code for like from a tutorial about indexing. Oh right, yeah. Yeah. [SPEAKER_TURN]
[00:17:21.640 --> 00:17:32.420]   Yeah, you can mess around some sort of nonsense, but yeah. The basics, is it? Okay, that's a good one. [SPEAKER_TURN]
[00:17:32.420 --> 00:17:38.180]   Yeah, that's handy. Yeah. So you think it shouldn't be too difficult? [SPEAKER_TURN]
[00:17:40.120 --> 00:17:57.640]   I don't know. I'm I was a bit worried about one well, when he talked about this with um file let's not with files. I mean how much do we actually need to do like extracting things from the X_M_L_ files and stuff to be able to do the indexing. [SPEAKER_TURN]
[00:17:57.640 --> 00:18:02.600]   Oh, I see. Yeah. You have to convert it to some sort of format. I suppose. [SPEAKER_TURN]
[00:18:02.600 --> 00:18:15.860]   Yeah. I've been trying to write something to read the X_M_L_ and get rid of it. And I can get rid of it, but I'm having trouble putting you anywhere else. So it'll come up on the screen. [SPEAKER_TURN]
[00:18:15.860 --> 00:18:18.080]   Mm-hmm. [SPEAKER_TURN]
[00:18:18.080 --> 00:18:25.080]   Back to whatever and jog it to play with it, but I guess I'm ready to put it. Yeah. Yeah. Yeah. [SPEAKER_TURN]
[00:18:26.100 --> 00:18:34.200]   I mean you should look at have a look at the the browser code then because, um you know, they already extract for example transcripts. [SPEAKER_TURN]
[00:18:34.200 --> 00:18:37.000]   Yeah, yeah, uh I'm sure they do. Yeah. [SPEAKER_TURN]
[00:18:37.000 --> 00:18:50.520]   I'm just gonna do yeah. Oh yeah. Oh well. I'm sure I'm sure I'm making a make of it. That's another question. I dunno. [SPEAKER_TURN]
[00:18:53.920 --> 00:19:08.380]   Um yeah, I guess I mean it was a good point we had that like collaborating with foul things. I mean if you already started them um and stuff, I guess we can [SPEAKER_TURN]
[00:19:08.380 --> 00:19:22.020]   Mm-hmm. [SPEAKER_TURN]
[00:19:23.000 --> 00:19:31.480]   what we need when we know more about what we need maybe. Yeah. Exactly. So you're still working on it. [SPEAKER_TURN]
[00:19:31.480 --> 00:19:46.400]   as is rather than the proper week, you know what I mean So. it's been a little bit manic the last couple of hours. [SPEAKER_TURN]
[00:19:46.400 --> 00:19:49.860]   Well you're an S_P_ and L_P_ and S_P_ too, right? Yeah, that's fine. [SPEAKER_TURN]
[00:19:50.020 --> 00:19:56.020]   we had to do an I_R_P_ whatever it was last week as well. Which is only on page one, but it's quite difficult to write a page. [SPEAKER_TURN]
[00:19:56.460 --> 00:20:26.300]   Yeah. Yeah. Oh yeah. Yeah, that's fine. But it was I had to sit there and think about it for a bit, but since it's back I don't know. Yeah, but S_P_R_P_ yeah, that's fine. Okay. So yeah, I was thinking that we can probably get uh start attacking this seriously. I'm just um you know, hopefully I'll have something by next week that we can sort of try and degrade it into the goo. I think that should be possible. [SPEAKER_TURN]
[00:20:26.440 --> 00:20:43.820]   The browsers I use it now or as I I don't know it, was there when I took it. Um it uses the um the N_X_T_ search to get the data out of the out of the files, which I think is odd, but I'm not quite sure how it works yet. I don't quite understand it. [SPEAKER_TURN]
[00:20:43.820 --> 00:20:46.320]   So money, money. [SPEAKER_TURN]
[00:20:46.320 --> 00:20:54.200]   To get the data, for example to get summarisation data, you have to search for I think the I_D_s on something.
[00:20:54.200 --> 00:20:55.780]   Yeah.
[00:20:56.420 --> 00:20:59.560]   That's pretty bizarre, isn't it? I guess that's right, where it's got the
[00:20:59.560 --> 00:21:02.440]   That is pretty bizarre. [SPEAKER_TURN]
[00:21:02.440 --> 00:21:09.040]   So they don't you know, they don't say look in that in that folder, but they say search everything. [SPEAKER_TURN]
[00:21:09.040 --> 00:21:11.980]   Yeah, it's pretty bizarre, yeah. [SPEAKER_TURN]
[00:21:11.980 --> 00:21:17.520]   Hmm. Well they did say it wasn't the most efficient. [SPEAKER_TURN]
[00:21:17.520 --> 00:21:30.300]   And it does load up quite quickly, sorry. That's not too much. But you know, it can have a look at like a window where you have to uh write a search query, or is it just what's behind it so, or where do you want it to go? [SPEAKER_TURN]
[00:21:30.300 --> 00:21:38.520]   Yeah. [SPEAKER_TURN]
[00:21:45.640 --> 00:21:47.000]   Text or whatever. [SPEAKER_TURN]
[00:21:47.000 --> 00:21:53.880]   For and it what it like it just has an open variable, transcript, list, whatever, find everything that's got that and then it'll open. [SPEAKER_TURN]
[00:21:53.880 --> 00:22:00.840]   So I don't know if you've looked at the search query language, but it's like a dollar A_ for D_ text, say. [SPEAKER_TURN]
[00:22:00.840 --> 00:22:05.660]   Yeah, 'cause that's the like the guy could he plays a variable, doesn't he? He could probably do that one. [SPEAKER_TURN]
[00:22:05.920 --> 00:22:21.600]   Yeah, because every every of all of these um these files have got um th an I_D_ and all the transcript files got the string, I don't know what it is, words or text or something in it. So just search just for those files. [SPEAKER_TURN]
[00:22:21.600 --> 00:22:22.780]   Okay. [SPEAKER_TURN]
[00:22:22.780 --> 00:22:23.420]   Mm-hmm. [SPEAKER_TURN]
[00:22:24.860 --> 00:22:38.240]   What do you like is there a open thingy or what are you like what what did you do to like how do you figure out that this user's mixed in search like, is it? [SPEAKER_TURN]
[00:22:38.240 --> 00:22:50.440]   Um well, it's in the code, just that it you know it yep, you define a search object and then you like search that string. I had to figure out how to how to do the same thing for the summary, that's how I [SPEAKER_TURN]
[00:22:53.360 --> 00:23:13.100]   understand how to do the transcript how they did the transcript to do the summary and so on. So yeah, just search for a specific string. So you could just, you know, you could use that when you if you wanna um take the data out. I was just thinking that, yeah. [SPEAKER_TURN]
[00:23:17.420 --> 00:23:47.000]   a question about what the c code means or where to find the code that you wanna you know whatever, you wanna do, then just send me an email. I can explain it. Yeah. Oh yeah. Actually in the moment it's got a load of rubbish because the the the browser that I used from this from you know that they gave us had a lot of extra stuff in and hadn't dared to throw an app to throw out all the functions that ha you know that I don't need. Yeah that's, exactly [SPEAKER_TURN]
[00:23:47.000 --> 00:23:55.980]   Yes. So they're all in there, yeah. Still. A lot of rubbish. But it will be get it will be chucked down by the end I think. [SPEAKER_TURN]
[00:23:55.980 --> 00:24:07.080]   Comming committing it out one bit at a time, running it to do it. Yeah. Yeah, that next. Good fun. [SPEAKER_TURN]
[00:24:07.080 --> 00:24:09.320]   Mm-hmm.
[00:24:09.320 --> 00:24:15.660]   So basically we got kind of this work maybe. Yeah that's, it. [SPEAKER_TURN]
[00:24:15.740 --> 00:24:25.660]   The odds to that isn't getting the long and the short with the long and the long. Yeah. Still at the bottom of the mountain actually. [SPEAKER_TURN]
[00:24:25.660 --> 00:24:29.000]   How many hours is this? [SPEAKER_TURN]
[00:24:29.000 --> 00:24:43.200]   Well should, you play the hippo to explore any other Yeah? How how's your um uh quantifier raising? Yeah. [SPEAKER_TURN]
[00:24:43.440 --> 00:24:47.600]   I don't remember what I'm supposed to do by the way. Am I supposed to do the summarisation thingy-bop? [SPEAKER_TURN]
[00:24:47.600 --> 00:24:49.360]   Mm-hmm.
[00:24:49.360 --> 00:24:54.420]   Mm-hmm. No. [SPEAKER_TURN]
[00:24:54.420 --> 00:24:56.220]   Who was doing it? [SPEAKER_TURN]
[00:24:56.220 --> 00:24:59.740]   You drew. [SPEAKER_TURN]
[00:24:59.740 --> 00:25:12.180]   Oh really? These can be changed. Um I don't know it, doesn't matter. If if you want to, I mean let's just you know, play around with some extra stuff, you have some time. Why not. [SPEAKER_TURN]
[00:25:12.300 --> 00:25:13.700]   It shouldn't be too different at all. [SPEAKER_TURN]
[00:25:13.700 --> 00:25:20.600]   So what courses should I have picked? Are we not doing anywhere? [SPEAKER_TURN]
[00:25:21.060 --> 00:25:47.940]   Well I'm doing communications of which we had one exercise and I've done that like two weeks ago. And that's in in two weeks. Yeah, it's uh nine nine courses on and ten or something. And then I'm doing um parallel programming languages in CISTAS. Which we had one exercise as well and that seemed like Friday.
[00:25:47.940 --> 00:25:59.420]   Well I'm not doing any back-hand meetings, I don't know. I'm going away with this topic. Yeah. [SPEAKER_TURN]
[00:25:59.480 --> 00:26:14.680]   Mm-hmm. I'll be happy when this week is over. Yeah. I don't know. I dunno. Is that about it though? Uh-huh. Anything else we need to discuss? [SPEAKER_TURN]
[00:26:14.680 --> 00:26:25.480]   Mm-hmm. [SPEAKER_TURN]
[00:26:25.480 --> 00:26:27.100]   Yeah. [SPEAKER_TURN]
[00:26:27.380 --> 00:26:31.060]   after a D_M_L_G_ or something and then we just have a little quick little update and just [SPEAKER_TURN]
[00:26:31.060 --> 00:26:39.960]   Yeah if you like. We can always decide then. Yeah. Yeah. Um so what what are you planning to do over the weekend then? [SPEAKER_TURN]
[00:26:39.960 --> 00:26:42.080]   Search. [SPEAKER_TURN]
[00:26:42.080 --> 00:26:48.220]   You're gonna do the search. You're gonna implement the search. Yeah. Yeah. Yeah. Yeah. [SPEAKER_TURN]
[00:26:48.220 --> 00:26:50.820]   Mm-hmm. [SPEAKER_TURN]
[00:26:51.120 --> 00:27:04.340]   version of, you know, the next query, etcetera. Um won't be the whole thing but, Um like we can just decide sort of the things that we do wanna do. Yeah. Well actually most of us just dialogue back. So I suppose we'll [SPEAKER_TURN]
[00:27:04.340 --> 00:27:13.920]   There's summary topic structure. [SPEAKER_TURN]
[00:27:13.920 --> 00:27:18.600]   Yeah. [SPEAKER_TURN]
[00:27:18.900 --> 00:27:25.340]   It's just that I think we need to set ourselves some milestones. Because otherwise, you know. [SPEAKER_TURN]
[00:27:25.340 --> 00:27:38.260]   Yeah, we get a high schedule. Yeah, pretty quick. Like you said. Yeah. Well actually do we wanna set a date for interim prototype? Specifically a date. [SPEAKER_TURN]
[00:27:40.820 --> 00:28:09.680]   Mm s well you decide I'm not very much of a person. The progression for is June twenty, two third of February. Um. Well how about we get it done by then I, mean. Then we can write about it in the in the interim report. You know, s you know, now we've got something to show. [SPEAKER_TURN]
[00:28:09.680 --> 00:28:10.260]   So then
[00:28:10.260 --> 00:28:24.120]   you can just look at the little deadlines. See Yeah. [SPEAKER_TURN]
[00:28:24.120 --> 00:28:43.980]   Um oh and the the search interface, maybe you should decide pretty pretty early what kind of things you wanna be able to search for. And then if you want you can give that to me and I'll I'll build you a nice, you know, a few tick boxes and drop down menus. [SPEAKER_TURN]
[00:28:43.980 --> 00:28:52.260]   Wasn't like what do you have now? Wasn't that pretty much a thing you did down? Or was it just like a similar one? [SPEAKER_TURN]
[00:28:52.260 --> 00:28:53.160]   That doesn't exist.
[00:28:54.160 --> 00:29:00.320]   That is completely cut and pasted. That does not exist Nicely. done then, isn't it? [SPEAKER_TURN]
[00:29:00.320 --> 00:29:03.900]   Yeah, it's just quite impressive actually. It's it's a little bit surprising. [SPEAKER_TURN]
[00:29:03.900 --> 00:29:14.960]   A screenshot of Google. [SPEAKER_TURN]
[00:29:14.960 --> 00:29:15.500]   Oh. [SPEAKER_TURN]
[00:29:15.500 --> 00:29:22.820]   That's not very bright Maybe. you should make it exist then. Yeah. [SPEAKER_TURN]
[00:29:23.140 --> 00:29:49.340]   It's gonna be a little difficult, more difficult. Yeah, I mean I was going to, but you know, the whole drop down menus and stuff, what we wanna search for is not quite agreed on, we no longer agreed on what we wanna be able to search for. So yeah, do you wanna, you know, just get together and think about the different types of search we could possibly do, and then and then we could discuss at next meeting which options we a really wanna include. [SPEAKER_TURN]
[00:29:49.340 --> 00:29:50.160]   Mm-hmm. [SPEAKER_TURN]
[00:29:51.360 --> 00:30:16.300]   I would probably involve researching the query language a bit as well to see what that can do, you know. It would be nice to uh meet up and you can sort of show us a bit, you know, already so we don't sort of have to do all the work again, you know, just like figuring out stuff. No, no already, what? Just like accessing the corpus and stuff. [SPEAKER_TURN]
[00:30:16.300 --> 00:30:18.680]   Mm-hmm. [SPEAKER_TURN]
[00:30:18.700 --> 00:30:33.480]   just sort of how just see the basics of what you've done. Like just get a a quick look over and just take a search tutorial and say yeah, or like like you know the class paths, all that sort of thing. Like just say okay well here's you know everything's basic also, and then just yeah, just jump right in. [SPEAKER_TURN]
[00:30:33.480 --> 00:30:36.800]   It's the query. Yeah. Yeah. [SPEAKER_TURN]
[00:30:36.800 --> 00:30:41.760]   Yeah. In the morning I, would say. [SPEAKER_TURN]
[00:30:41.760 --> 00:30:48.060]   Well there's some I_M_D_ taking the class. By the by the way, you can't do any of that. [SPEAKER_TURN]
[00:30:48.060 --> 00:31:02.860]   The whole thing is, you know, that yeah, something like that could happen any time 'cause uh yeah, like my schedule is like a class break, class break, class, you know, sorta I_C_M_C_. So yeah, I know exactly, exactly. That's what I'm thinking. So something like that, and then if I have a class I can run off and do whatever. [SPEAKER_TURN]
[00:31:02.960 --> 00:31:12.200]   Time. Um yeah, maybe a minute or so. Yeah. What Monday? Forty six? Yeah, this Monday Just. [SPEAKER_TURN]
[00:31:12.200 --> 00:31:22.560]   Sure. Sure. [SPEAKER_TURN]
[00:31:22.560 --> 00:31:24.760]   It's possible. [SPEAKER_TURN]
[00:31:24.760 --> 00:31:28.160]   It really is alright. Yeah, well is there one time to do that or do you forward a little? [SPEAKER_TURN]
[00:31:28.160 --> 00:31:28.960]   It's forward. [SPEAKER_TURN]
[00:31:28.960 --> 00:31:31.220]   Unless you wanna do it absol um after. [SPEAKER_TURN]
[00:31:31.560 --> 00:31:33.860]   Yeah, that's an option too. Hmm? I'm drinking. [SPEAKER_TURN]
[00:31:33.860 --> 00:31:35.600]   Sure. [SPEAKER_TURN]
[00:31:35.600 --> 00:31:39.960]   Three is good though. Um but we haven't thought well, what was your what was your fair? [SPEAKER_TURN]
[00:31:39.960 --> 00:31:45.420]   Or three thirty. I just yeah. I think it's necessary, necessary. [SPEAKER_TURN]
[00:31:45.420 --> 00:32:01.540]   Three thirty in the in the computer room in here, and then we just have to go. If you can't can you get in the front door? Oh, we haven't got a key doo. Can none of you get in? Yeah, go on, find out. The c the c this computer room's in here, the M_S_C_ computer room. Right, is in there. Well do you [SPEAKER_TURN]
[00:32:01.560 --> 00:32:13.180]   me, I'll show you where our letters are all in there and we can all go in there. Yeah, exactly. You do need to have a key for it, like that's the thing that's going on the whole button. Yeah, somebody like that. [SPEAKER_TURN]
[00:32:13.180 --> 00:32:19.400]   The computer room. [SPEAKER_TURN]
[00:32:19.400 --> 00:32:19.740]   Okay. [SPEAKER_TURN]
[00:32:19.740 --> 00:32:28.200]   Yeah, it's only got it's crap, it's only got about four or five computers in there. It would do quite a good for our job 'cause it can all sit in there, it's not very ergonomic, we have people in there. Right. [SPEAKER_TURN]
[00:32:28.200 --> 00:32:29.640]   Is there a printer in there too? [SPEAKER_TURN]
[00:32:29.760 --> 00:32:45.540]   Yeah, it's right in there. And there's a photocopier with the code on it, yeah I think. It's cool yeah, Yeah. I think we could put it at your uh assignment looking, for what to do. Yeah. All the comments, I'm thinking. [SPEAKER_TURN]
[00:32:45.540 --> 00:32:59.020]   Mm-hmm. [SPEAKER_TURN]
[00:32:59.300 --> 00:33:05.420]   well, she doesn't call late. Hopefully it's not that man's, though it, might matter. [SPEAKER_TURN]
[00:33:06.200 --> 00:33:35.020]   Actually, it's not even confirmed yet. What? She doesn't even know what she's talking about. Maybe it doesn't happen at all. It's really quick, next week Should. we? Is it really? Well, it's supposed to be Next week. It's weak seems to be weak. I don't think we have a reading week now. I don't know. We have one class at least, when we like speech perception we, don't have any lectures. But I think we're having like all other classes. So I don't think there was an official one. [SPEAKER_TURN]
[00:33:35.020 --> 00:33:36.180]   an official one.
[00:33:36.260 --> 00:33:38.800]   Last semester, I think it was just like I called her, wasn't it? [SPEAKER_TURN]
[00:33:38.800 --> 00:34:04.380]   Um something like I think one of a couple classes I cancelled. Yeah. I think it goes up to the professor whether they did it or not. Well, I don't feel like he yeah, okay. Yeah. I think yeah. I think D_I_L_ will probably have one to make up for missing classes. D_I_L_ will probably have classes. Yeah. [SPEAKER_TURN]
[00:34:04.580 --> 00:34:06.620]   Just gonna have the last week. [SPEAKER_TURN]
[00:34:06.620 --> 00:34:09.320]   Okay. That's me Monday. [SPEAKER_TURN]
[00:34:09.320 --> 00:34:10.640]   Done? [SPEAKER_TURN]
[00:34:10.640 --> 00:34:20.500]   Yeah. Um I guess. That's it. Was there anything else? Do anyone know anyone? Why not? Like take care of this or should I take care of it? [SPEAKER_TURN]
[00:34:20.500 --> 00:34:21.240]   Okay. [SPEAKER_TURN]
[00:34:21.240 --> 00:34:22.580]   Oh, we trust you. Yeah. [SPEAKER_TURN]
[00:34:22.580 --> 00:34:39.980]   I don't know if there was something Oh. I, think he basically said um the same things as in the comments. If I could leave it. Maybe that? [SPEAKER_TURN]
[00:34:39.980 --> 00:34:40.780]   Yeah. [SPEAKER_TURN]
[00:34:40.780 --> 00:34:41.980]   Um
[00:34:41.980 --> 00:34:47.200]   there's really one multiple Windows from a single app. [SPEAKER_TURN]
[00:34:47.200 --> 00:34:49.620]   From a single app. [SPEAKER_TURN]
[00:34:49.620 --> 00:34:52.480]   But most will probably want to go with defaults.
[00:34:52.480 --> 00:34:58.780]   Yeah. Well, I think that was the same thing. I think the comments the basically the same as what he says. [SPEAKER_TURN]
[00:34:58.780 --> 00:35:04.980]   Yeah, exactly. Just the other comment about whether you wanna do both speaker characterisation and topic segmentation. [SPEAKER_TURN]
[00:35:04.980 --> 00:35:06.080]   Mm-hmm. [SPEAKER_TURN]
[00:35:06.080 --> 00:35:08.680]   Okay, that's probably it. I think I'm gonna say it. [SPEAKER_TURN]
[00:35:08.680 --> 00:35:22.460]   Yeah. And now we have seven percent of the persons? Go. So we need to t be okay both, right? If you're okay with uh having uh something [SPEAKER_TURN]
[00:35:22.460 --> 00:35:42.480]   Yeah, yeah. Should we turn this up or no? Is she still here? Yes, you are.
'''                                                    
                                                                                                                
parsed_segments = parse_input_text(input_text)                                                                  
rttm_output = generate_rttm(parsed_segments)                                                                    
                                                                                                                
print(rttm_output)                                           

SPEAKER EN2002a 1 0.000 3.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 3.000 24.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 29.000 5.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 34.000 2.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 36.000 0.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 36.000 3.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 39.000 4.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 43.000 2.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 45.000 4.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 49.000 1.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 50.000 16.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 66.000 2.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 88.000 15.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 103.000 3.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 136.000 7.000 <NA> <NA> speaker0 <NA> <NA>
SPEAKER EN2002a 1 143.000 8.000 <NA> <NA> speaker1 <NA> <NA>
SPEAKER EN2002a 1 151.000 1.000 <NA>

In [11]:
# once again manually remove lines where the duration of a segment is 0.000
!python3 /home/kozi/Documents/_onlab_git/dscore/score.py --collar 0.100  -s "/home/kozi/Documents/_onlab_git/ami/whisper_cpp_tdrz_rttm.rttm" -r "/home/kozi/Documents/_onlab_git/ami/ami_dataset/test/rttm/EN2002a.rttm"
# 80%, (85% with --ignore_overlaps) interesting that the same model can give different outputs (though probably the whisper model used here is stronger that whatever exact solution the true tinydiarize library was using)

Loading speaker turns from reference RTTMs...
Loading speaker turns from system RTTMs...
Trimming reference speaker turns to UEM scoring regions...
Trimming system speaker turns to UEM scoring regions...
Checking for overlapping reference speaker turns...
Checking for overlapping system speaker turns...
Scoring...
File               DER    JER    B3-Precision    B3-Recall    B3-F1    GKT(ref, sys)    GKT(sys, ref)    H(ref|sys)    H(sys|ref)    MI    NMI
---------------  -----  -----  --------------  -----------  -------  ---------------  ---------------  ------------  ------------  ----  -----
EN2002a          80.00  86.82            0.15         0.41     0.22             0.03             0.01          3.21          1.38  0.05   0.02
*** OVERALL ***  80.00  86.82            0.15         0.41     0.22             0.03             0.01          3.21          1.38  0.05   0.02
